# Capstone Project 1: Data Wrangling 1

Data source: https://www.yelp.com/dataset. The first dataset used includes 192,609 businesses with over 1.2 million business attributes like hours, parking, availability, and ambience. The second dataset used includes the aggregated check-ins over time for each of the 192,609 businesses. (According to Yelp)

Data wrangling activities:
- Read json files.
- Format json strings.
- Create pandas dataframes from json strings.
- Filter businesses into restaurants dataframe.
- Filter restaurants outliers - those with less than 10 reviews.
- Triage restaurants into those of good, moderate and poor standing based on stars into 3 restaurant dataframes.
- Triage checkins by restaurants standing (good, moderate, poor) into 3 restaurant checkin dataframes.
- Parse and reshape date columns of restaurant checkin dataframes.
- Sort and reset index for all resulting dataframes before using for analysis.

Data wrangling result:
- 1 dataframe containing restaurants in good standing (4 or more stars).
- 1 dataframe containing restaurants in moderate standing (between 2 and 4 stars).
- 1 dataframe containing restaurants in poor standing (less than 2 stars).
- 1 dataframe containing checkins of restaurants in good standing with date column parsed and reshaped.
- 1 dataframe containing checkins of restaurants in moderate standing with date column parsed and reshaped.
- 1 dataframe containing checkins of restaurants in poor standing with date column parsed and reshaped.

In [1]:
import pandas as pd
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read file into list
with open('dataset/business.json','r',encoding='utf8') as f:
    businesses = f.readlines()
with open('dataset/checkin.json','r',encoding='utf8') as f:
    checkins = f.readlines()

# Remove the trailing "\n" from each line
businesses = list(map(lambda x: x.rstrip(), businesses))
checkins = list(map(lambda x: x.rstrip(), checkins))

# Prepare json string
bjson = '['+','.join(businesses)+']'
cjson = '['+','.join(checkins)+']'

# Load json string into pandas dataframe
dfb = pd.read_json(bjson)
dfc = pd.read_json(cjson)

In [2]:
# Create boolean array for filtering for Restaurants
isrest = dfb.categories.str.contains('Restaurants') & pd.notnull(dfb.categories)

# Filter the businesses dataframe with boolean array and assign to restaurants dataframe
restaurants = dfb[isrest]


In [3]:
# Exploratory data analysis
print(dfc.shape) # There is no checkin data for 30659 of the businesses
print(restaurants.shape) # 59371 out of the 192609 businesses are restaurants
print(restaurants.columns)

(161950, 2)
(59371, 14)
Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')


In [4]:
# Filter out restaurants with less than 10 reviews.
restaurants = restaurants[restaurants.review_count > 10]

# Triage restaurants into 3 categories of standing using star ratings. Good, moderate and poor. Assign to dataframes.
goodr = restaurants[restaurants.stars >= 4]
poorr = restaurants[restaurants.stars <= 2]
modr = restaurants[restaurants.stars < 4]
modr = modr[modr.stars > 2]

In [5]:
print(goodr.shape)
print(modr.shape)
print(poorr.shape)


(16156, 14)
(21233, 14)
(2241, 14)


In [6]:
# Create boolean arrays for filtering for restaurant checkins
isgoodrc = dfc['business_id'].isin(goodr['business_id'])
ismodrc = dfc['business_id'].isin(modr['business_id'])
ispoorrc = dfc['business_id'].isin(poorr['business_id'])

# Filter the checkins dataframe with boolean arrays and assign to new dataframes
goodrc = dfc[isgoodrc]
modrc = dfc[ismodrc]
poorrc = dfc[ispoorrc]

print(goodrc.shape)
print(modrc.shape)
print(poorrc.shape)

(16103, 2)
(21156, 2)
(2230, 2)


In [7]:
goodrc.head(4)

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
5,--9e1ONYQuAa-CB_Rrw7Tw,"2010-02-08 05:56:47, 2010-02-15 04:47:42, 2010..."
10,--FBCX-N37CMYDfs790Bnw,"2010-05-31 07:57:10, 2010-05-31 07:58:37, 2010..."
26,--cZ6Hhc9F7VkKXxHMVZSQ,"2010-04-24 19:48:58, 2010-05-01 19:28:23, 2010..."


In [8]:
# Import datetime parse date column in checkin dataframes
import datetime as dt

# Create function for parsing dates and reshaping checkin dataframe. 
# Returns new dataframe with one row for each checkin date of each restaurant.
def transform_df(df):
    rclist = []
    for idx,row in df.iterrows():
        dates = row['date'].split(',')
        for d in dates:
            rcdict = {}
            rcdict.update({'business_id':row['business_id'],'date':dt.datetime.strptime(d.strip(),'%Y-%m-%d %H:%M:%S')})
            rclist.append(rcdict)
    rcdf = pd.DataFrame(rclist)
    return rcdf

In [9]:
# Call function on each checkin dataframe and assign result to new dataframe.
goodr_checkins = transform_df(goodrc)
modr_checkins = transform_df(modrc)
poorr_checkins = transform_df(poorrc)

In [10]:
# Sort restaurants dataframes by stars in descending order and reset index
goodr = goodr.sort_values('stars',ascending=False).reset_index(drop=True)
modr = modr.sort_values('stars',ascending=False).reset_index(drop=True)
poorr = poorr.sort_values('stars',ascending=False).reset_index(drop=True)


In [164]:
# Create user defined function to pivot out the attributes we want to analyze
def pivot_attributes(akeys,df):
    for k in akeys:
        avals = [d.get(k) if d else None for d in df['attributes']]
        df[k] = avals
    return df

In [165]:
attribute_keys = ['RestaurantsReservations','WiFi','HasTV','RestaurantsDelivery','RestaurantsTakeout',
                  'RestaurantsGoodForGroups','OutdoorSeating','Alcohol','RestaurantsPriceRange2',
                  'GoodForKids','NoiseLevel','RestaurantsAttire']
for k in attribute_keys:
    goodr[k] = ''
    modr[k] = ''
    poorr[k] = ''

In [166]:
goodr = pivot_attributes(attribute_keys,goodr)

In [167]:
modr = pivot_attributes(attribute_keys,modr)
poorr = pivot_attributes(attribute_keys,poorr)

In [176]:
# Create user defined function to clean up values in certain attribute columns
def clean_values(cols,df):
    for c in cols:
        newvals = [v[2:-1] if (v and v[0] == 'u') else v[1:-1] if (v and v[0] == "'") else v if v else None for v in df[c]]
        df[c] = newvals
    return df


In [177]:
cols = ['Alcohol','NoiseLevel','RestaurantsAttire']
goodr = clean_values(cols,goodr)

In [179]:
modr = clean_values(cols,modr)
poorr = clean_values(cols,modr)

In [180]:
%store goodr
%store modr
%store poorr
%store goodr_checkins
%store modr_checkins
%store poorr_checkins
%store restaurants

Stored 'goodr' (DataFrame)
Stored 'modr' (DataFrame)
Stored 'poorr' (DataFrame)
Stored 'goodr_checkins' (DataFrame)
Stored 'modr_checkins' (DataFrame)
Stored 'poorr_checkins' (DataFrame)
Stored 'restaurants' (DataFrame)
